# 모델 매개변수 최적화하기

- 데이터에 매개변수를 최적화하여 모델을 학습하고, 검증, 테스트
- epoch를 통해 반복 단계에서 모델을 출력을 추측하고, 추측과 정답 사이의 오류(손실)을 계산

In [17]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)


In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits



# 하이퍼파라미터

- epoch : 데이터셋을 반복하는 횟수
- batch size : 매개 변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
- learning rate : 각 배치/에폭에서 모델의 매개변수를 조절하는 비율, 값이 작을수록 학습속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생

In [8]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# 최적화 단계
- 하이퍼 파라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화 함
- 학습 단계 : 학습용 데이터셋을 반복하고 최적의 매개변수를 수렴
- 검증/테스트 단계 : 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋 반복

# 손실 함수
- 손실 함수를 통해 획득한 결과 값과 실제 값 사이의 틀린정도를 측정하여 그 값을 최소화하는 역활을 함
- 회귀문제 : nn.MSELoss(평균 제곱 오차)
- 분류문제 : nn.NLLLoss(음의 로그 우도), nn.LogSoftmax, nn.CrossEntropyLoss

In [9]:
# 손실 함수 초기화
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저
- 각 학습 단계에서 오류를 줄이기 위해 모델 매개변수를 조정하는 과정
- 다양한 옵티마이저 https://pytorch.org/docs/stable/optim.html

In [10]:
model = NeuralNetwork()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
def train_loop(dataloader, model, loss_fn, optimier):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    # Compute prediction and loss
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()


  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

  

In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------
loss: 2.310990  [    0/60000]
loss: 2.295753  [ 6400/60000]
loss: 2.274568  [12800/60000]
loss: 2.264747  [19200/60000]
loss: 2.247291  [25600/60000]
loss: 2.215755  [32000/60000]
loss: 2.223442  [38400/60000]
loss: 2.183763  [44800/60000]
loss: 2.183394  [51200/60000]
loss: 2.152907  [57600/60000]
Test Error: 
 Accuracy: 47.5%, Avg loss: 2.146639 

Epoch 2
-------------------------
loss: 2.158990  [    0/60000]
loss: 2.151841  [ 6400/60000]
loss: 2.084055  [12800/60000]
loss: 2.106554  [19200/60000]
loss: 2.050786  [25600/60000]
loss: 1.981230  [32000/60000]
loss: 2.018080  [38400/60000]
loss: 1.926248  [44800/60000]
loss: 1.931486  [51200/60000]
loss: 1.870603  [57600/60000]
Test Error: 
 Accuracy: 54.2%, Avg loss: 1.863859 

Epoch 3
-------------------------
loss: 1.892254  [    0/60000]
loss: 1.869815  [ 6400/60000]
loss: 1.742311  [12800/60000]
loss: 1.795844  [19200/60000]
loss: 1.680395  [25600/60000]
loss: 1.628356  [32000/60000]
loss: 1.663210